In [77]:
%matplotlib
import pandas as pd
import itertools
import datetime as dt
from datetime import datetime
import numpy as np
import time
from scipy.optimize import linear_sum_assignment
from matplotlib import pyplot as plt
from scipy.interpolate import pchip

t_start = time.time()
df1_original = pd.read_csv('/Users/mac/Box Sync/Bike-Ride-Sharing/data_code/st9_12months/st9_2015_dpdata(sample)_new/st9_3.csv')
df1 = df1_original
read_col = pd.read_csv('/Users/mac/Box Sync/Bike-Ride-Sharing/st9_dp_matching/matching3.csv')
col_ind = read_col['header'].tolist()
df1['matching'] = pd.Series(col_ind)

Using matplotlib backend: MacOSX


In [78]:
df1['day'] = pd.Series(0, index = df1.index)
df1['combined'] = pd.Series(False, index = df1.index)
df1['period'] = pd.Series(0, index = df1.index)

In [79]:
i = 0
day_list = []
while i < len(df1.index):
    time_day = datetime.strptime(df1.loc[i]['dp_datetime'], "%Y-%m-%d %H:%M:%S")
    day = time_day.date().day
    hour = time_day.time().hour
    minute = time_day.time().minute
    if day not in day_list: day_list.append(day)
    df1.loc[i,'day'] = day
    if col_ind[i] != i:
        df1.loc[i,'combined'] = True
    if hour == 7:
        if minute < 11: df1.loc[i,'period'] = 1
        elif minute < 21: df1.loc[i,'period'] = 2
        elif minute < 31: df1.loc[i,'period'] = 3
        elif minute < 41: df1.loc[i,'period'] = 4
        elif minute < 51: df1.loc[i,'period'] = 5
        else: df1.loc[i,'period'] = 6
    else:
        if minute < 11: df1.loc[i,'period'] = 7
        elif minute < 21: df1.loc[i,'period'] = 8
        elif minute < 31: df1.loc[i,'period'] = 9
        elif minute < 41: df1.loc[i,'period'] = 10
        elif minute < 51: df1.loc[i,'period'] = 11
        else: df1.loc[i,'period'] = 12
    i = i + 1

df1_sorted = df1.sort_values(['dp_datetime'])
day_list = sorted(day_list)

dayratio_list = []

In [80]:
bike_demand = []
for day_temp in day_list:
    bike_demand_temp = []
    for period_temp in range(1,13):
        total = np.sum(df1_sorted[(df1_sorted['day'] == day_temp)&(df1_sorted['period'] == period_temp)]['passenger_count'].tolist())
        bike_demand_temp.append(total)
    bike_demand.append(bike_demand_temp)

In [83]:
bike_demand

[[23, 11, 22, 12, 34, 10, 29, 21, 33, 29, 29, 29],
 [26, 35, 41, 29, 39, 34, 34, 25, 34, 40, 29, 26],
 [33, 13, 31, 21, 34, 36, 32, 36, 30, 38, 27, 25],
 [35, 16, 39, 33, 28, 37, 47, 17, 32, 24, 20, 29],
 [16, 19, 32, 17, 34, 20, 21, 31, 24, 24, 26, 20],
 [28, 27, 32, 22, 25, 36, 27, 30, 38, 30, 20, 26],
 [32, 31, 30, 27, 39, 37, 15, 27, 29, 27, 16, 42],
 [23, 16, 20, 22, 35, 38, 37, 12, 15, 25, 32, 43],
 [29, 15, 33, 17, 29, 25, 25, 25, 29, 43, 33, 15],
 [27, 21, 22, 23, 15, 32, 37, 33, 19, 28, 35, 14],
 [30, 32, 20, 39, 22, 20, 12, 28, 38, 21, 33, 17],
 [30, 35, 20, 26, 48, 24, 29, 24, 30, 28, 22, 24],
 [36, 40, 34, 35, 26, 36, 50, 39, 41, 27, 48, 53],
 [36, 21, 36, 28, 18, 31, 33, 30, 37, 41, 38, 32],
 [24, 27, 24, 25, 45, 46, 42, 30, 17, 38, 24, 25],
 [33, 33, 18, 30, 30, 26, 43, 28, 28, 33, 27, 24],
 [13, 21, 15, 31, 15, 44, 43, 35, 43, 43, 39, 27],
 [24, 12, 38, 27, 32, 27, 31, 30, 49, 36, 33, 26],
 [50, 33, 22, 48, 23, 42, 27, 40, 38, 27, 29, 28],
 [67, 23, 37, 43, 33, 30, 41, 2

In [84]:
allbike_demand = []
for item in bike_demand:
    allbike_demand = allbike_demand + item

import plotly.plotly as py
from matplotlib import colors as mcolors

x = np.array(range(1,265))
y1 = allbike_demand
y2 = np.empty(len(x))
y2.fill(35)

In [87]:
np.max(y1)

67

In [88]:
plt.plot(x, y1, '-', linewidth=2, label='Required Bikes',color='#4682b4')
plt.plot(x, y2, '-', linewidth=2, label='Capacity of Station9',color='red')

plt.fill_between(x, y1,np.zeros(len(x)), facecolor='#4682b4', interpolate=True)

locs, labels = plt.xticks()

plt.xticks(np.arange(0,278,12),[0]+ day_list)
plt.xlim(0,264)
plt.ylim(0,70)

(0, 70)

In [89]:
plt.xlabel(r'Day in March')
plt.ylabel('Number of Required Bikes at Hub')
plt.legend(loc='upper left')